In [1]:
using DataStructures

In [2]:
function find_path(graph, visited = [], start_point = (1,2), target_point = (23,22))
    if start_point == target_point
        return 0
    end

    max_val = 0

    for a_next in keys(graph[(start_point)]["next"])
        if !(a_next in visited)
            new_visited = deepcopy(visited)
            push!(new_visited, a_next)
            max_val = max(max_val, find_path(graph, new_visited, a_next, target_point) + graph[(start_point)]["next"][a_next])
        end
    end
        
    return max_val
end

find_path (generic function with 4 methods)

In [3]:
test_dict = Dict((4, 12) => Dict("next" => Dict((12, 22) => 30, (14, 14) => 24), "type" => '.'), (1, 2) => Dict("next" => Dict((6, 4) => 15), "type" => '.'), (12, 22) => Dict("next" => Dict((20, 20) => 10), "type" => '.'), (23, 22) => Dict("next" => Dict(), "type" => '.'), (20, 14) => Dict("next" => Dict((20, 20) => 10), "type" => '.'), (14, 14) => Dict("next" => Dict((20, 14) => 10, (12, 22) => 18), "type" => '.'), (20, 20) => Dict("next" => Dict((23, 22) => 5), "type" => '.'), (6, 4) => Dict("next" => Dict((1, 2) => 15, (14, 6) => 22, (4, 12) => 22), "type" => '.'), (14, 6) => Dict("next" => Dict((20, 14) => 38, (14, 14) => 12), "type" => '.'))

Dict{Tuple{Int64, Int64}, Dict{String, Any}} with 9 entries:
  (1, 2)   => Dict("next"=>Dict((6, 4)=>15), "type"=>'.')
  (20, 14) => Dict("next"=>Dict((20, 20)=>10), "type"=>'.')
  (14, 6)  => Dict("next"=>Dict((20, 14)=>38, (14, 14)=>12), "type"=>'.')
  (12, 22) => Dict("next"=>Dict((20, 20)=>10), "type"=>'.')
  (4, 12)  => Dict("next"=>Dict((12, 22)=>30, (14, 14)=>24), "type"=>'.')
  (14, 14) => Dict("next"=>Dict((20, 14)=>10, (12, 22)=>18), "type"=>'.')
  (20, 20) => Dict("next"=>Dict((23, 22)=>5), "type"=>'.')
  (23, 22) => Dict("next"=>Dict{Any, Any}(), "type"=>'.')
  (6, 4)   => Dict("next"=>Dict((1, 2)=>15, (14, 6)=>22, (4, 12)=>22), "type"=>…

In [4]:
test_dict[(6,4)]

Dict{String, Any} with 2 entries:
  "next" => Dict((1, 2)=>15, (14, 6)=>22, (4, 12)=>22)
  "type" => '.'

In [5]:
find_path(test_dict)

94

In [6]:
(long_path) = open("23_1.txt") do f
    l_path = 0
    graph = Dict()
    # Start by reading everything
    l = readline(f)
    start = []
    target = (141,140)
    nb_rows = 0
    nb_cols = 0
    for (idx_c, c) in enumerate(collect(l))
        if c == '.'
            graph[(1, idx_c)] = Dict("type" => c, "next" => DefaultDict(Int))
            start = (1, idx_c)
            break
        end
    end
    for (idx_r, l) in enumerate(eachline(f))
        for (idx_c, c) in enumerate(collect(l))
            if c != '#'
                graph[(idx_r+1, idx_c)] = Dict("type" => c, "next" => DefaultDict(Int))
            end
            nb_cols = idx_c
        end
        nb_rows = idx_r
    end

    # Find all neighbours
    for (r,c) in keys(graph)
        if (r-1,c) in keys(graph)
            # push!(graph[(r,c)]["next"], (r-1,c))
            graph[(r,c)]["next"][(r-1,c)] = graph[(r,c)]["type"] == 'v' ? 10001 : 1
        end
        if (r+1,c) in keys(graph)
            # push!(graph[(r,c)]["next"], (r+1,c))
            graph[(r,c)]["next"][(r+1,c)] = graph[(r,c)]["type"] == '^' ? 10001 : 1
        end
        if (r,c-1) in keys(graph)
            #push!(graph[(r,c)]["next"], (r,c-1))
            # println("($r,$c) gives")
            graph[(r,c)]["next"][(r,c-1)] = graph[(r,c)]["type"] == '>' ? 10001 : 1
        end
        if (r,c+1) in keys(graph)
            #push!(graph[(r,c)]["next"], (r,c+1))
            graph[(r,c)]["next"][(r,c+1)] = graph[(r,c)]["type"] == '<' ? 10001 : 1
        end
    end

    all_keys = keys(graph)

    
    #println("Wait")
    #println("Wait")
    #tries = [(1,2),(4,11), (4,12), (6,5)]
    #for a_try in tries
    #    println("$a_try gives $(graph[a_try])")
    #end
    #println("Wait")
    #println("Wait")

    for a_point in all_keys
        if (length(graph[a_point]["next"]) == 2)
            l_ = pop!(graph[a_point]["next"])
            r_ = pop!(graph[a_point]["next"]) # This guy is empty now!
            l = l_[1]
            l_dist = l_[2]
            r = r_[1]
            r_dist = r_[2]
            delete!(graph[l]["next"], a_point)
            delete!(graph[r]["next"], a_point)
            # deleteat!(graph[l]["next"], findall(x->x==a_point,graph[l]["next"]))
            # deleteat!(graph[r]["next"], findall(x->x==a_point,graph[r]["next"]))
            push!(graph[l]["next"], [r, r_dist < 10000 ? ((r_dist+l_dist) % 10000) : r_dist+l_dist])
            push!(graph[r]["next"], [l, l_dist < 10000 ? ((r_dist+l_dist) % 10000) : r_dist+l_dist])
            delete!(graph, a_point)
        end
        #for a in keys(graph)
        #    if a[1] == a_point[1]
        #        println("Current $a is $(graph[a])\n\n")
        #    end
        #end
    end



    for a_node in keys(graph)
        for a_neighbour in keys(graph[a_node]["next"])
            # println(graph[a_node]["next"][a_neighbour])
            if graph[a_node]["next"][a_neighbour] > 10000
                # println("Deleting from $ ")
                delete!(graph[a_node]["next"], a_neighbour)
            end
            
        end
    end
            
    
    # println(graph)
       
    (find_path(graph, [], start, target))

    
end

2106

In [7]:
function find_path2(graph, visited = [(1,2)], start_point = (1,2), target_point = (23,22))
    if start_point == target_point
        return 0
    end
    #println("visited is $visited")
    nexts = graph[(start_point)]["next"]
    #println("Current point is $start_point with $nexts")

    max_val = -1000

    for a_next in keys(nexts)
        if !(a_next in visited)
            new_visited = deepcopy(visited)
            push!(new_visited, a_next)
            max_val = max(max_val, find_path2(graph, new_visited, a_next, target_point) + graph[(start_point)]["next"][a_next])
        end
    end
        
    return max_val
end

find_path2 (generic function with 4 methods)

In [16]:
(long_path) = open("23_1.txt") do f
    l_path = 0
    graph = Dict()
    # Start by reading everything
    l = readline(f)
    start = []
    target = (141,140)
    nb_rows = 0
    nb_cols = 0
    for (idx_c, c) in enumerate(collect(l))
        if c == '.'
            graph[(1, idx_c)] = Dict("type" => c, "next" => DefaultDict(Int))
            start = (1, idx_c)
            break
        end
    end
    for (idx_r, l) in enumerate(eachline(f))
        for (idx_c, c) in enumerate(collect(l))
            if c != '#'
                graph[(idx_r+1, idx_c)] = Dict("type" => c, "next" => DefaultDict(Int))
            end
            nb_cols = idx_c
        end
        nb_rows = idx_r
    end

    # Find all neighbours
    for (r,c) in keys(graph)
        if (r-1,c) in keys(graph)
            # push!(graph[(r,c)]["next"], (r-1,c))
            graph[(r,c)]["next"][(r-1,c)] = 1
        end
        if (r+1,c) in keys(graph)
            # push!(graph[(r,c)]["next"], (r+1,c))
            graph[(r,c)]["next"][(r+1,c)] = 1
        end
        if (r,c-1) in keys(graph)
            #push!(graph[(r,c)]["next"], (r,c-1))
            # println("($r,$c) gives")
            graph[(r,c)]["next"][(r,c-1)] = 1
        end
        if (r,c+1) in keys(graph)
            #push!(graph[(r,c)]["next"], (r,c+1))
            graph[(r,c)]["next"][(r,c+1)] = 1
        end
    end

    all_keys = keys(graph)


    for a_point in all_keys
        if (length(graph[a_point]["next"]) == 2)
            l_ = pop!(graph[a_point]["next"])
            r_ = pop!(graph[a_point]["next"]) # This guy is empty now!
            l = l_[1]
            l_dist = l_[2]
            r = r_[1]
            r_dist = r_[2]
            delete!(graph[l]["next"], a_point)
            delete!(graph[r]["next"], a_point)
            # deleteat!(graph[l]["next"], findall(x->x==a_point,graph[l]["next"]))
            # deleteat!(graph[r]["next"], findall(x->x==a_point,graph[r]["next"]))
            push!(graph[l]["next"], [r, r_dist+l_dist])
            push!(graph[r]["next"], [l, r_dist+l_dist])
            delete!(graph, a_point)
        end
        #for a in keys(graph)
        #    if a[1] == a_point[1]
        #        println("Current $a is $(graph[a])\n\n")
        #    end
        #end
    end

            
    
    println(graph)
    # (0)   
    (find_path2(graph, [start], start, target))

    
end

Dict{Any, Any}((64, 82) => Dict{String, Any}("next" => DefaultDict{Any, Any, DataType}((36, 88) => 182, (54, 112) => 144, (62, 66) => 70, (88, 80) => 90), "type" => '.'), (88, 80) => Dict{String, Any}("next" => DefaultDict{Any, Any, DataType}((84, 68) => 40, (110, 88) => 142, (80, 106) => 174, (64, 82) => 90), "type" => '.'), (12, 106) => Dict{String, Any}("next" => DefaultDict{Any, Any, DataType}((6, 88) => 96, (44, 124) => 610, (38, 100) => 156), "type" => '.'), (1, 2) => Dict{String, Any}("next" => DefaultDict{Any, Any, DataType}((20, 20) => 165), "type" => '.'), (141, 140) => Dict{String, Any}("next" => DefaultDict{Any, Any, DataType}((134, 136) => 23), "type" => '.'), (100, 136) => Dict{String, Any}("next" => DefaultDict{Any, Any, DataType}((80, 138) => 98, (134, 136) => 182, (106, 108) => 182), "type" => '.'), (80, 106) => Dict{String, Any}("next" => DefaultDict{Any, Any, DataType}((80, 138) => 212, (54, 112) => 144, (106, 108) => 148, (88, 80) => 174), "type" => '.'), (106, 10) 

6350

In [9]:
println("6401 is not the right answer")

6401 is not the right answer


In [10]:
full_test = Dict((4, 12) => Dict("next" => Dict((12, 22) => 30, (14, 14) => 24, (6, 4) => 22), "type" => '.'), (1, 2) => Dict("next" => Dict((6, 4) => 15), "type" => '.'), (12, 22) => Dict("next" => Dict((4, 12) => 30, (20, 20) => 10, (14, 14) => 18), "type" => '.'), (23, 22) => Dict("next" => Dict((20, 20) => 5), "type" => '.'), (20, 14) => Dict("next" => Dict((14, 6) => 38, (20, 20) => 10, (14, 14) => 10), "type" => '.'), (14, 14) => Dict("next" => Dict((20, 14) => 10, (14, 6) => 12, (12, 22) => 18, (4, 12) => 24), "type" => '.'), (20, 20) => Dict("next" => Dict((20, 14) => 10, (12, 22) => 10, (23, 22) => 5), "type" => '.'), (6, 4) => Dict("next" => Dict((1, 2) => 15, (14, 6) => 22, (4, 12) => 22), "type" => '.'), (14, 6) => Dict("next" => Dict((20, 14) => 38, (14, 14) => 12, (6, 4) => 22), "type" => '.'))

Dict{Tuple{Int64, Int64}, Dict{String, Any}} with 9 entries:
  (1, 2)   => Dict("next"=>Dict((6, 4)=>15), "type"=>'.')
  (20, 14) => Dict("next"=>Dict((14, 6)=>38, (20, 20)=>10, (14, 14)=>10), "type…
  (14, 6)  => Dict("next"=>Dict((20, 14)=>38, (14, 14)=>12, (6, 4)=>22), "type"…
  (12, 22) => Dict("next"=>Dict((4, 12)=>30, (20, 20)=>10, (14, 14)=>18), "type…
  (4, 12)  => Dict("next"=>Dict((12, 22)=>30, (14, 14)=>24, (6, 4)=>22), "type"…
  (14, 14) => Dict("next"=>Dict((20, 14)=>10, (14, 6)=>12, (12, 22)=>18, (4, 12…
  (20, 20) => Dict("next"=>Dict((20, 14)=>10, (12, 22)=>10, (23, 22)=>5), "type…
  (23, 22) => Dict("next"=>Dict((20, 20)=>5), "type"=>'.')
  (6, 4)   => Dict("next"=>Dict((1, 2)=>15, (14, 6)=>22, (4, 12)=>22), "type"=>…

In [11]:
find_path(full_test, [], (1,2), (23,22))

154

In [12]:
true_test = Dict((64, 82) => Dict("next" => Dict((36, 88) => 182, (54, 112) => 144, (62, 66) => 70, (88, 80) => 90), "type" => '.'), (88, 80) => Dict("next" => Dict((84, 68) => 40, (110, 88) => 142, (80, 106) => 174, (64, 82) => 90), "type" => '.'), (12, 106) => Dict("next" => Dict((6, 88) => 96, (44, 124) => 610, (38, 100) => 156), "type" => '.'), (1, 2) => Dict("next" => Dict((20, 20) => 165), "type" => '.'), (141, 140) => Dict("next" => Dict((134, 136) => 23), "type" => '.'), (100, 136) => Dict("next" => Dict((80, 138) => 98, (134, 136) => 182, (106, 108) => 182), "type" => '.'), (80, 106) => Dict("next" => Dict((80, 138) => 212, (54, 112) => 144, (106, 108) => 148, (88, 80) => 174), "type" => '.'), (106, 10) => Dict("next" => Dict((90, 6) => 104, (110, 34) => 140, (138, 34) => 380), "type" => '.'), (132, 56) => Dict("next" => Dict((104, 62) => 202, (138, 34) => 112, (134, 84) => 190), "type" => '.'), (106, 108) => Dict("next" => Dict((110, 88) => 120, (80, 106) => 148, (132, 114) => 148, (100, 136) => 182), "type" => '.'), (104, 62) => Dict("next" => Dict((84, 68) => 106, (110, 88) => 188, (110, 34) => 146, (132, 56) => 202), "type" => '.'), (58, 132) => Dict("next" => Dict((80, 138) => 132, (44, 124) => 122, (54, 112) => 96), "type" => '.'), (44, 124) => Dict("next" => Dict((58, 132) => 122, (38, 100) => 122, (12, 106) => 610), "type" => '.'), (134, 136) => Dict("next" => Dict((141, 140) => 23, (132, 114) => 160, (100, 136) => 182), "type" => '.'), (54, 36) => Dict("next" => Dict((66, 18) => 98, (82, 36) => 156, (62, 66) => 214, (32, 38) => 128), "type" => '.'), (40, 14) => Dict("next" => Dict((66, 18) => 266, (20, 20) => 170, (32, 38) => 112), "type" => '.'), (132, 114) => Dict("next" => Dict((134, 136) => 160, (106, 108) => 148, (134, 84) => 248), "type" => '.'), (6, 88) => Dict("next" => Dict((36, 88) => 182, (20, 62) => 228, (12, 106) => 96), "type" => '.'), (84, 68) => Dict("next" => Dict((104, 62) => 106, (82, 36) => 210, (62, 66) => 92, (88, 80) => 40), "type" => '.'), (20, 62) => Dict("next" => Dict((6, 88) => 228, (42, 60) => 100, (6, 30) => 258), "type" => '.'), (6, 30) => Dict("next" => Dict((20, 62) => 258, (20, 20) => 112, (32, 38) => 126), "type" => '.'), (66, 18) => Dict("next" => Dict((90, 6) => 164, (54, 36) => 98, (40, 14) => 266), "type" => '.'), (80, 138) => Dict("next" => Dict((58, 132) => 132, (80, 106) => 212, (100, 136) => 98), "type" => '.'), (134, 84) => Dict("next" => Dict((110, 88) => 156, (132, 114) => 248, (132, 56) => 190), "type" => '.'), (62, 66) => Dict("next" => Dict((84, 68) => 92, (54, 36) => 214, (42, 60) => 90, (64, 82) => 70), "type" => '.'), (38, 100) => Dict("next" => Dict((36, 88) => 50, (54, 112) => 76, (44, 124) => 122, (12, 106) => 156), "type" => '.'), (90, 6) => Dict("next" => Dict((66, 18) => 164, (82, 36) => 194, (106, 10) => 104), "type" => '.'), (32, 38) => Dict("next" => Dict((54, 36) => 128, (42, 60) => 136, (6, 30) => 126, (40, 14) => 112), "type" => '.'), (20, 20) => Dict("next" => Dict((1, 2) => 165, (6, 30) => 112, (40, 14) => 170), "type" => '.'), (42, 60) => Dict("next" => Dict((36, 88) => 154, (20, 62) => 100, (62, 66) => 90, (32, 38) => 136), "type" => '.'), (110, 34) => Dict("next" => Dict((104, 62) => 146, (82, 36) => 178, (138, 34) => 172, (106, 10) => 140), "type" => '.'), (36, 88) => Dict("next" => Dict((6, 88) => 182, (64, 82) => 182, (42, 60) => 154, (38, 100) => 50), "type" => '.'), (110, 88) => Dict("next" => Dict((106, 108) => 120, (104, 62) => 188, (134, 84) => 156, (88, 80) => 142), "type" => '.'), (54, 112) => Dict("next" => Dict((58, 132) => 96, (80, 106) => 144, (64, 82) => 144, (38, 100) => 76), "type" => '.'), (138, 34) => Dict("next" => Dict((110, 34) => 172, (132, 56) => 112, (106, 10) => 380), "type" => '.'), (82, 36) => Dict("next" => Dict((84, 68) => 210, (90, 6) => 194, (54, 36) => 156, (110, 34) => 178), "type" => '.'))

Dict{Tuple{Int64, Int64}, Dict{String, Any}} with 36 entries:
  (6, 88)    => Dict("next"=>Dict((36, 88)=>182, (20, 62)=>228, (12, 106)=>96),…
  (1, 2)     => Dict("next"=>Dict((20, 20)=>165), "type"=>'.')
  (90, 6)    => Dict("next"=>Dict((66, 18)=>164, (82, 36)=>194, (106, 10)=>104)…
  (54, 36)   => Dict("next"=>Dict((66, 18)=>98, (82, 36)=>156, (62, 66)=>214, (…
  (110, 34)  => Dict("next"=>Dict((104, 62)=>146, (82, 36)=>178, (138, 34)=>172…
  (132, 114) => Dict("next"=>Dict((134, 136)=>160, (106, 108)=>148, (134, 84)=>…
  (6, 30)    => Dict("next"=>Dict((20, 62)=>258, (20, 20)=>112, (32, 38)=>126),…
  (36, 88)   => Dict("next"=>Dict((6, 88)=>182, (64, 82)=>182, (42, 60)=>154, (…
  (132, 56)  => Dict("next"=>Dict((104, 62)=>202, (138, 34)=>112, (134, 84)=>19…
  (44, 124)  => Dict("next"=>Dict((58, 132)=>122, (38, 100)=>122, (12, 106)=>61…
  (20, 20)   => Dict("next"=>Dict((1, 2)=>165, (6, 30)=>112, (40, 14)=>170), "t…
  (62, 66)   => Dict("next"=>Dict((84, 68)=>92, (54, 36)=>214, (4

In [13]:
find_path2(full_test)

154

In [14]:
find_path2(true_test, [(1,2)], (1,2), (141,140))

6350